In [8]:
#Imports
import numpy as np

from scipy.stats import bootstrap
from scipy.optimize import curve_fit
from scipy.stats import moyal

import matplotlib.patches as mpatches
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mplhep as hep

from datetime import date
import time
import warnings

In [15]:
'''
A lot of functions

'''

def length_of_true_segs(segs):
    
    true_drift = []

    for each_seg in segs:
        
        true_dq = each_seg['n_electrons']/1e3
        true_dx = each_seg['dx']
        
        t_drift = abs(each_seg['t'])
        
        x = each_seg['x']
        
        z = each_seg['z']
        
        true_drift.append([true_dq,t_drift,x,z, true_dx])
        
    return true_drift

def dQdx_per_TPC(dQdx, t_drift, x_mid, z_mid, mask_cut):
    mask_cut = mask_cut

    dQdx_masked = dQdx[mask_cut]
    t_drift_masked = t_drift[mask_cut]  

    return dQdx_masked, t_drift_masked

def dQdx_median_or_landau(dQdx, t_drifted, bins):
    
    median_dQdx_points = []
    landau_dQdx_points = []
    
    t_points = []
    
    amount_per_bin = []
    
    median_error = []
    landau_error = []
    for i in range(len(bins)-1):
        mask_cut = (t_drifted >= bins[i]) & (t_drifted < bins[i+1])
        
        dQdx_masked = dQdx[mask_cut]
        
        amount_per_bin.append(len(dQdx_masked))
    
        median_dQdx_points.append(np.median(dQdx_masked))
        
        popt_landau, pcov_landau = fit_landau(dQdx_masked)
        landau_dQdx_points.append(popt_landau[1])
        
        t_points.append((bins[i] + bins[i+1])/2)
        
        boot_strap = bootstrapp((dQdx_masked,))
        
        median_error.append(boot_strap.confidence_interval)
        landau_error.append(pcov_landau[1][1])
    return median_dQdx_points, landau_dQdx_points, t_points, median_error, amount_per_bin, landau_error

def bootstrapp(data):
    return bootstrap(data, np.median, n_resamples = 10, confidence_level = 0.95, method = 'percentile')

def func(x, A, τ):
    return  A * np.exp(-x/τ)

def error_bars(error, points):
    high_error_dq=[]
    low_error_dq=[]
    
    for i in range(len(error)):
        high_error_dq.append(error[i][1]-points[i])
        low_error_dq.append(error[i][0]-points[i])
        
    asymmetric_error = [abs(np.array(low_error_dq)), abs(np.array(high_error_dq))] 
    
    return asymmetric_error

def median_electron_lifetime_graph(fig, ax, new_dq_points, true_new_dq_points, t_array, true_t_array, popt, true_popt, x, y, true_y, asymmetric_rec_dq, asymmetric_true_dq, number_tpc, amount, amount2):
    number = [[1,0],[3,1],[5,2],[7,3],[2,0],[4,1],[6,2],[8,3]]
    Mod = [0,1,2,3]
    warnings.filterwarnings("ignore")
    #warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib.legend")
    
    #Label TPCs and modules
    if number_tpc < 4:
        number_tpc1 = number[number_tpc][1]
        j = 0
    
    if number_tpc >= 4:
        number_tpc1 = number[number_tpc][1]
        j = 1
    
    if (j == 0) & (number[number_tpc][0] == 1):
        TPC = 'TPC'+'{}'.format(number[number_tpc][0])
        Module = 'Module'+'{}'.format(0)
        
        ax[j, number_tpc1].text(1, 1.05, TPC, horizontalalignment='right', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #TPC
        ax[j, number_tpc1].text(0.5, 1.1, Module, horizontalalignment='center', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #Module
        
    
    elif (j == 0) & (number[number_tpc][0] == 3):
        TPC = 'TPC'+'{}'.format(number[number_tpc][0])
        Module = 'Module'+'{}'.format(1)
        
        ax[j, number_tpc1].text(1, 1.05, TPC, horizontalalignment='right', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #TPC
        ax[j, number_tpc1].text(0.5, 1.1, Module, horizontalalignment='center', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #Module
    
    elif (j == 0) & (number[number_tpc][0] == 5):
        TPC = 'TPC'+'{}'.format(number[number_tpc][0])
        Module = 'Module'+'{}'.format(2)
        
        ax[j, number_tpc1].text(1, 1.05, TPC, horizontalalignment='right', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #TPC
        ax[j, number_tpc1].text(0.5, 1.1, Module, horizontalalignment='center', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #Module
    
    elif (j == 0) & (number[number_tpc][0] == 7):
        TPC = 'TPC'+'{}'.format(number[number_tpc][0])
        Module = 'Module'+'{}'.format(3)
        
        ax[j, number_tpc1].text(1, 1.05, TPC, horizontalalignment='right', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #TPC
        ax[j, number_tpc1].text(0.5, 1.1, Module, horizontalalignment='center', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #Module
    
    else:
        TPC = 'TPC'+'{}'.format(number[number_tpc][0])
        
        ax[j, number_tpc1].text(1, 1.05, TPC, horizontalalignment='right', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #TPC
        
    
    ax[j, number_tpc1].set_xlim(0, high_value)
    ax[j, number_tpc1].set_ylim(45,70)
    simulated_legend = mpatches.Patch(label='sim $e^{} lifetime, 2.2  [ms]$'.format('-'))
    #recon
    recon_points = ax[j,number_tpc1].scatter(t_array, new_dq_points, s= 250, marker = '_', c = 'b', label = '$e^{}$ lifetime, {} $\pm$ {} [ms]'.format('-',round(popt[1],2), round(rec_e_life_error,2)))
    ax[j,number_tpc1].plot(x, y, c = 'r')
    ax[j,number_tpc1].errorbar(t_array,new_dq_points, yerr = asymmetric_rec_dq, c = 'b', fmt= 'none')
    #true
    true_points = ax[j,number_tpc1].scatter(true_t_array, true_new_dq_points, s= 250, marker = '_', c = 'lime',label = '$e^{}$ lifetime using true segs, {} $\pm$ {} [ms]'.format('-',round(true_popt[1],2), round(true_e_life_error,2)))
    ax[j,number_tpc1].plot(x, true_y, c = 'k')
    ax[j,number_tpc1].errorbar(true_t_array,true_new_dq_points, yerr = asymmetric_true_dq, c = 'lime',fmt = 'none')

    ax[j,number_tpc1].legend(handles = [simulated_legend, recon_points, true_points],fontsize = 9, loc = 'lower left')
    
    for i, txt in enumerate(amount):
        ax[j,number_tpc1].annotate(txt, (t_array[i], new_dq_points[i]+.5), size = 8, rotation = 90)
    for i, txt in enumerate(amount2):
        ax[j,number_tpc1].annotate(txt, (true_t_array[i], true_new_dq_points[i]+.5), size = 8, rotation = 90)
     
    if (number_tpc == 7):
        fig.savefig('/global/homes/d/demaross/MR4graphs/final_Electron_lifetime_MR4_{}.png'.format(date.today()))
    
        fig.show()
       
    return fig, TPC

def landau_electron_lifetime_graph(fig, ax, new_dq_points, true_new_dq_points, t_array, true_t_array, popt, true_popt, x, y, true_y, asymmetric_rec_dq, asymmetric_true_dq, number_tpc, amount, amount2):
    number = [[1,0],[3,1],[5,2],[7,3],[2,0],[4,1],[6,2],[8,3]]
    Mod = [0,1,2,3]
    warnings.filterwarnings("ignore")
    #warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib.legend")
    
    #Label TPCs and modules
    if number_tpc < 4:
        number_tpc1 = number[number_tpc][1]
        j = 0
    
    if number_tpc >= 4:
        number_tpc1 = number[number_tpc][1]
        j = 1
    
    if (j == 0) & (number[number_tpc][0] == 1):
        TPC = 'TPC'+'{}'.format(number[number_tpc][0])
        Module = 'Module'+'{}'.format(0)
        
        ax[j, number_tpc1].text(1, 1.05, TPC, horizontalalignment='right', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #TPC
        ax[j, number_tpc1].text(0.5, 1.1, Module, horizontalalignment='center', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #Module
        
    
    elif (j == 0) & (number[number_tpc][0] == 3):
        TPC = 'TPC'+'{}'.format(number[number_tpc][0])
        Module = 'Module'+'{}'.format(1)
        
        ax[j, number_tpc1].text(1, 1.05, TPC, horizontalalignment='right', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #TPC
        ax[j, number_tpc1].text(0.5, 1.1, Module, horizontalalignment='center', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #Module
    
    elif (j == 0) & (number[number_tpc][0] == 5):
        TPC = 'TPC'+'{}'.format(number[number_tpc][0])
        Module = 'Module'+'{}'.format(2)
        
        ax[j, number_tpc1].text(1, 1.05, TPC, horizontalalignment='right', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #TPC
        ax[j, number_tpc1].text(0.5, 1.1, Module, horizontalalignment='center', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #Module
    
    elif (j == 0) & (number[number_tpc][0] == 7):
        TPC = 'TPC'+'{}'.format(number[number_tpc][0])
        Module = 'Module'+'{}'.format(3)
        
        ax[j, number_tpc1].text(1, 1.05, TPC, horizontalalignment='right', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #TPC
        ax[j, number_tpc1].text(0.5, 1.1, Module, horizontalalignment='center', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #Module
    
    else:
        TPC = 'TPC'+'{}'.format(number[number_tpc][0])
        
        ax[j, number_tpc1].text(1, 1.05, TPC, horizontalalignment='right', verticalalignment='center', transform=ax[j, number_tpc1].transAxes) #TPC
        
    
    ax[j, number_tpc1].set_xlim(0, high_value)
    ax[j, number_tpc1].set_ylim(40,60)
    simulated_legend = mpatches.Patch(label='sim $e^{} lifetime, 2.2  [ms]$'.format('-'))
    #recon
    recon_points = ax[j,number_tpc1].scatter(t_array, new_dq_points, s= 250, marker = '_', c = 'b', label = '$e^{}$ lifetime, {} $\pm$ {} [ms]'.format('-',round(popt[1],2), round(rec_e_life_error,2)))
    ax[j,number_tpc1].plot(x, y, c = 'r')
    ax[j,number_tpc1].errorbar(t_array,new_dq_points, yerr = asymmetric_rec_dq, c = 'b', fmt= 'none')
    #true
    true_points = ax[j,number_tpc1].scatter(true_t_array, true_new_dq_points, s= 250, marker = '_', c = 'lime',label = '$e^{}$ lifetime using true segs, {} $\pm$ {} [ms]'.format('-',round(true_popt[1],2), round(true_e_life_error,2)))
    ax[j,number_tpc1].plot(x, true_y, c = 'k')
    ax[j,number_tpc1].errorbar(true_t_array,true_new_dq_points, yerr = asymmetric_true_dq, c = 'lime',fmt = 'none')
    
    ax[j,number_tpc1].legend(handles = [simulated_legend, recon_points, true_points],fontsize = 9, loc = 'lower left')
    '''
    for i, txt in enumerate(amount):
        ax[j,number_tpc1].annotate(txt, (t_array[i], new_dq_points[i]+.5), size = 8, rotation = 90)
    for i, txt in enumerate(amount2):
        ax[j,number_tpc1].annotate(txt, (true_t_array[i], true_new_dq_points[i]+.5), size = 8, rotation = 90)
    ''' 
    if (number_tpc == 7):
        fig.savefig('/global/homes/d/demaross/MR4graphs/final_Electron_lifetime_MR4_landau_{}.png'.format(date.today()))
    
        fig.show()
       
    return fig, TPC

def dQdx_plot(recon_dQdx, true_dQdx, recon_t_drifted, true_t_drifted, bins, time_bins, TPC):
    fig1, ax1 = plt.subplots(figsize = (10,10), nrows= 2, ncols = 2)
    
    plt.style.use(hep.style.CMS)

    plt.rcParams.update({'font.size': 12})
    
    #warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib.legend")
    for i in range(len(time_bins)-1):
        recon_mask_cut = (recon_t_drifted/10000 >= time_bins[i]) & (recon_t_drifted/10000 < time_bins[i+1])
        
        recon_dQdx_masked = recon_dQdx[recon_mask_cut]
        
        true_mask_cut = (true_t_drifted/1000 >= time_bins[i]) & (true_t_drifted/1000 < time_bins[i+1])
        
        true_dQdx_masked = true_dQdx[true_mask_cut]
        if i < 8:
            h, bins = np.histogram(recon_dQdx_masked, bins = bins)
            hep.histplot(h,stack = True, bins= bins, ax = ax1[0,0], label = 't_drift {} to {} [ms]'.format(round(time_bins[i],3),round(time_bins[i+1],3)))
            

            h, bins = np.histogram(true_dQdx_masked, bins = bins)
            hep.histplot(h,stack = True, bins= bins, ax= ax1[0,1], label = 't_drift {} to {} [ms]'.format(round(time_bins[i],3),round(time_bins[i+1],3)))
            #print(np.median(true_dQdx_masked))
        else:
            h, bins = np.histogram(recon_dQdx_masked, bins = bins)

            hep.histplot(h,stack = True, bins= bins, ax = ax1[1,0], label = 't_drift {} to {} [ms]'.format(round(time_bins[i],3),round(time_bins[i+1],3)))


            h, bins = np.histogram(true_dQdx_masked, bins = bins)
   
            hep.histplot(h,stack = True, bins= bins, ax= ax1[1,1], label = 't_drift {} to {} [ms]'.format(round(time_bins[i],3),round(time_bins[i+1],3)))
            
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            ax1[0,0].legend(fontsize = 7)
            ax1[0,1].legend(fontsize = 7)
            ax1[1,0].legend(fontsize = 7)
            ax1[1,1].legend(fontsize = 7)

        ax1[0,0].set_title('Recon dQdx per t_drift bin')
        ax1[0,1].set_title('True dQdx per t_drift bin')
        ax1[1,0].set_title('Recon dQdx per t_drift bin')
        ax1[1,1].set_title('True dQdx per t_drift bin')
        
        fig1.suptitle(TPC)
        fig1.supxlabel('$dQ/dx (ke^{-}/cm)$')
        fig1.supylabel('Counts')
        fig1.savefig('/global/homes/d/demaross/MR4graphs/final_{}_dQdx_per_time_bin_MR4_{}.png'.format(TPC,date.today()))
        plt.close(fig1)

    return fig1

def fit_landau(dqdx):
    # Generate some sample data
    np.random.seed(0)
    data = dqdx  # Generate Gaussian-distributed data
    data = data[data > 0]  # Remove negative values

    # Define the Landau PDF function
    def landau_pdf(x, mu, c):
        return moyal.pdf(x, c, mu)

    initial_guess = (55, 1)  # Initial guess for parameters (mu, c)
    bounds = ([0, 0], [np.inf, np.inf])  # Lower and upper bounds for parameters
    popt, pcov = curve_fit(landau_pdf, data, np.zeros_like(data) + 1, p0=initial_guess, bounds=bounds)
    #print(popt, pcov[1][1])
    
    return popt, pcov

In [10]:
'''

Load the reconstructed and true segments.

'''


recon_segs = np.load('/global/homes/d/demaross/MR4graphs/t_drift_segs_m4_final_2024-05-03.npy')
all_segs = np.load('/global/homes/d/demaross/MR4graphs/true_rock_muon_segs_final_m4_2024-04-30.npy')

segment_length_cut = 0

mask = recon_segs[5] >= segment_length_cut

t_drift_segsss = recon_segs[0][mask]

all_x_mid_a = recon_segs[1][mask]
all_y_mid_a = recon_segs[2][mask]
all_z_mid_a = recon_segs[3][mask]

all_dQ_a = recon_segs[4][mask]
all_dx_a = recon_segs[5][mask]

true_stuff = length_of_true_segs(all_segs) 

true_stuff_array = np.array(true_stuff)

# Note index 0 is dq, index 1 is time drifted, index 2 is x, index 3 is z, and index 4 is dx
mask = true_stuff_array[:,4] >= segment_length_cut

true_stuff_array = true_stuff_array[mask]

In [11]:
#For MR5
'''
x_boundaries = np.array([-63.931, -3.069, 3.069, 63.931])
y_boundaries = np.array([-42-19.8543, -42+103.8543])
z_boundaries = np.array([-64.3163,  -2.6837, 2.6837, 64.3163])
'''

#For MR4
x_boundaries = np.array([-63.931, -3.069, 3.069, 63.931])
y_boundaries = np.array([-42-19.8543, -42+103.8543]) - 268
z_boundaries = np.array([-64.3163,  -2.6837, 2.6837, 64.3163]) +1300
# x bins for TPC "1 and 2, Module 1"  j = 2 / z >_bound[2] and for TPC "5 and 6, Module 3" j =1 / z < z_bound[1]
bins1_5 = [x_boundaries[0]*-1, (x_boundaries[1] + x_boundaries[0]) * 0.5*-1]
bins2_6 = [(x_boundaries[1] + x_boundaries[0]) * 0.5*-1, x_boundaries[1]*-1]

# x bins for TPC "3 and 4, MOdule 1"  j = 2 / z >_bound[2] and for TPC "7 and 8, Module 3" j =1  / z < z_bound[1]
bins4_8 = [x_boundaries[0],(x_boundaries[0] + x_boundaries[1]) * 0.5]
bins3_7 = [(x_boundaries[0] + x_boundaries[1]) * 0.5, x_boundaries[1]]

In [12]:
'''
This BLock of code mask all of the segments into their individual TPCs

'''

mask_cut_1 = (all_x_mid_a <= bins1_5[0]) & (all_x_mid_a > bins1_5[1]) & (all_z_mid_a > z_boundaries[2])

mask_cut_3 = (all_x_mid_a >= bins3_7[0]) & (all_x_mid_a < bins3_7[1]) & (all_z_mid_a > z_boundaries[2])

mask_cut_5 = (all_x_mid_a <= bins1_5[0]) & (all_x_mid_a > bins1_5[1]) & (all_z_mid_a < z_boundaries[1])

mask_cut_7 = (all_x_mid_a >=bins3_7[0]) & (all_x_mid_a < bins3_7[1]) & (all_z_mid_a < z_boundaries[1])

mask_cut_2 = (all_x_mid_a <= bins2_6[0]) & (all_x_mid_a > bins2_6[1]) & (all_z_mid_a > z_boundaries[2])

mask_cut_4 = (all_x_mid_a >=bins4_8[0]) & (all_x_mid_a < bins4_8[1]) & (all_z_mid_a > z_boundaries[2])

mask_cut_6 = (all_x_mid_a <= bins2_6[0]) & (all_x_mid_a > bins2_6[1]) & (all_z_mid_a < z_boundaries[1])

mask_cut_8 = (all_x_mid_a >=bins4_8[0]) & (all_x_mid_a < bins4_8[1]) & (all_z_mid_a < z_boundaries[1])

all_cuts = [mask_cut_1, mask_cut_3, mask_cut_5, mask_cut_7, mask_cut_2, mask_cut_4, mask_cut_6, mask_cut_8]


t_mask_cut_1 = (true_stuff_array[:,2] <= bins1_5[0]) & (true_stuff_array[:,2] > bins1_5[1]) & (true_stuff_array[:,3] > z_boundaries[2])

t_mask_cut_3 = (true_stuff_array[:,2] >= bins3_7[0]) & (true_stuff_array[:,2] < bins3_7[1]) & (true_stuff_array[:,3] < z_boundaries[2])

t_mask_cut_5 = (true_stuff_array[:,2] <= bins1_5[0]) & (true_stuff_array[:,2] > bins1_5[1]) & (true_stuff_array[:,3] < z_boundaries[1])

t_mask_cut_7 = (true_stuff_array[:,2] >=bins3_7[0]) & (true_stuff_array[:,2] < bins3_7[1]) & (true_stuff_array[:,3] < z_boundaries[1])

t_mask_cut_2 = (true_stuff_array[:,2] <= bins2_6[0]) & (true_stuff_array[:,2] > bins2_6[1]) & (true_stuff_array[:,3] > z_boundaries[2])

t_mask_cut_4 = (true_stuff_array[:,2] >=bins4_8[0]) & (true_stuff_array[:,2] < bins4_8[1]) & (true_stuff_array[:,3] > z_boundaries[2])

t_mask_cut_6 = (true_stuff_array[:,2] <= bins2_6[0]) & (true_stuff_array[:,2] > bins2_6[1]) & (true_stuff_array[:,3] < z_boundaries[1])

t_mask_cut_8 = (true_stuff_array[:,2] >=bins4_8[0]) & (true_stuff_array[:,2] < bins4_8[1]) & (true_stuff_array[:,3] < z_boundaries[1])
t_all_cuts = [t_mask_cut_1, t_mask_cut_3, t_mask_cut_5, t_mask_cut_7, t_mask_cut_2, t_mask_cut_4, t_mask_cut_6, t_mask_cut_8]

In [ ]:
'''

Calculate the Electron Lifetime and create plots.

'''
fig, ax = plt.subplots(nrows = 2, ncols = 4, figsize = (20,10))
fig2, ax2 = plt.subplots(nrows = 2, ncols = 4, figsize = (20,10))

plt.style.use(hep.style.CMS)
plt.rcParams.update({'font.size': 12})

#recon_amount_segs_per_tpc = []
#true_amount_segs_per_tpc = []

n_bins = 15
#values for time in microseconds
low_value = 0.035
high_value = 0.175
#TIme bins for reconstructed (time is in ticks)
time_bins = np.linspace(low_value, high_value,n_bins)

binsss = np.linspace(0,200,100)

for number_tpc in range(len(all_cuts)):
    
    warnings.simplefilter("ignore")
    mask_cut = all_cuts[number_tpc]
    true_mask_cut = t_all_cuts[number_tpc]
    
    recon_new_dq, recon_new_t = dQdx_per_TPC(all_dQ_a/all_dx_a, t_drift_segsss, all_x_mid_a, all_z_mid_a, mask_cut)

    true_new_dq, true_new_t = dQdx_per_TPC(true_stuff_array[:,0]/true_stuff_array[:,4], true_stuff_array[:,1], true_stuff_array[:,2], true_stuff_array[:,3], true_mask_cut)

    #Recon
    recon_median_dq_points, recon_landau_dq_points, t_points, recon_median_error, amount , recon_landau_error  = dQdx_median_or_landau(recon_new_dq, recon_new_t/10000, time_bins) #Time divided by 10000 because units are off
    
    #time stuff
    t_array = np.array(t_points)
    mask = (t_array > low_value) & (t_array < high_value)
    t_array = t_array[mask]
    #recon_amount_segs_per_tpc.append(amount)
    
    time_binned_recon_median_dq_points, time_binned_recon_landau_dq_points = np.array(recon_median_dq_points)[mask], np.array(recon_landau_dq_points)[mask]    
    
    #true
    true_median_dq_points, true_landau_dq_points, true_new_t_points, true_median_error, amount2, true_landau_error = dQdx_median_or_landau(true_new_dq, true_new_t/1000, time_bins)
 
    #true_amount.append(amount2)
    
    #Time stuff
    true_t_array = np.array(true_new_t_points)
    mask = (true_t_array > low_value) & (true_t_array < high_value)
    true_t_array = true_t_array[mask]
    
    #Dq stuff
    time_binned_true_median_dq_points, time_binned_true_landau_dq_points = np.array(true_median_dq_points)[mask], np.array(true_landau_dq_points)[mask]
    
    #get error bars
    asymmetric_rec_dq = error_bars(recon_median_error, time_binned_recon_median_dq_points)
    asymmetric_true_dq = error_bars(true_median_error,time_binned_true_median_dq_points)
    
    #recon
    xdata = t_array
    recon_median_ydata = time_binned_recon_median_dq_points

    #true
    true_xdata = true_t_array
    true_median_ydata = time_binned_true_median_dq_points

    
    popt, pcov = curve_fit(func, xdata, recon_median_ydata, p0 =  [np.max(recon_median_dq_points), 2.2], sigma = (asymmetric_rec_dq[0]+asymmetric_rec_dq[1])/2)
    true_popt, true_pcov = curve_fit(func, true_xdata, true_median_ydata, p0 = [np.max(true_median_dq_points), 2.2], sigma = (asymmetric_true_dq[0]+asymmetric_true_dq[1])/2)
    
    rec_e_life_error = round(np.sqrt(np.diag(pcov))[1],2)
    true_e_life_error = round(np.sqrt(np.diag(true_pcov))[1],2)
    '''
    
    run this for landau. Uncomment popt,pcov and true_popt, true_pcov below and comment out the two above
    
    '''
    
    landau_popt, landau_pcov = curve_fit(func, xdata, time_binned_recon_landau_dq_points, p0 =  [np.max(recon_landau_dq_points), 2.2], sigma = recon_landau_error)
    landau_true_popt, landau_true_pcov = curve_fit(func, true_xdata, time_binned_true_landau_dq_points, p0 = [np.max(true_landau_dq_points), 2.2], sigma = true_landau_error)
    
    #X-points for electron lifetime plot
    x = np.linspace(0,high_value, 200)

    #Y-ponts for electron lifetime plot
    median_y = popt[0]*np.exp(-x/(popt[1]))
    true_median_y = true_popt[0]*np.exp(-x/(true_popt[1]))
    
    #Y-ponts for electron lifetime plot
    landau_y = landau_popt[0]*np.exp(-x/(landau_popt[1]))
    landau_true_y = landau_true_popt[0]*np.exp(-x/(landau_true_popt[1]))
    
    fig, TPC = median_electron_lifetime_graph(fig, ax, time_binned_recon_median_dq_points, time_binned_true_median_dq_points, t_array, true_t_array, popt, true_popt, x, median_y, true_median_y, asymmetric_rec_dq, asymmetric_true_dq, number_tpc,amount, amount2)

    fig1 = dQdx_plot(recon_new_dq, true_new_dq, recon_new_t, true_new_t, binsss, time_bins, TPC)
    
    '''
    run this fig to get landau points instead of median, Uncomment below and comment out fig,tpc above
    '''
    fig2, TPC = landau_electron_lifetime_graph(fig2, ax2, time_binned_recon_landau_dq_points, time_binned_true_landau_dq_points, t_array, true_t_array, landau_popt, landau_true_popt, x, landau_y, landau_true_y, recon_landau_error, true_landau_error, number_tpc,amount, amount2)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that 